In [1]:
#          ,  ,
#          \\ \\           
#          ) \\ \\    _p_ 
#          )^\))\))  /  *\ 
#           \_|| || / /^`-'  MOD 3
#  __       -\ \\--/ /     PROJECT
#<'  \\___/   ___. )'     FLATIRON
#     `====\ )___/\\        SCHOOL
#          //     `"          DATA
#          \\    /  \      SCIENCE
#          `"

In [2]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [3]:
#import pandas and datetime

import pandas as pd
import datetime
import numpy as np

In [4]:
# This is the truncated version of the larger file
df = pd.read_csv('health inspection.csv')

In [5]:
# For some reason it added the index back into dataframe, so I removed it
df.drop('Unnamed: 0', axis=1, inplace = True)

In [6]:
# This allows us to see the entire text in dataframe cells
pd.set_option('display.max_colwidth', -1)

In [7]:
# This creates a reference dataframe so we can join it back if we want to
df_violations = df[['CAMIS', 'VIOLATION CODE', 'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'inspection_date']]

In [8]:
#demonstration of the violations dataframe

df_violations.head(2)

,CAMIS,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,inspection_date
0,30075445,04L,Evidence of mice or live mice present in facility's food and/or non-food areas.,Critical,2019-05-16
1,30075445,08A,Facility not vermin proof. Harborage or conditions conducive to attracting vermin to the premises and/or allowing vermin to exist.,Not Critical,2019-05-16


In [9]:
df['ACTION'].unique()

array(['Violations were cited in the following area(s).',
       'Establishment re-opened by DOHMH',
       'Establishment re-closed by DOHMH',
       'Establishment Closed by DOHMH.  Violations were cited in the following area(s) and those requiring immediate action were addressed.',
       'No violations were recorded at the time of this inspection.'],
      dtype=object)

In [16]:
# export the df_violations df to a csv to lock it in between Chris and I 
# df_violations.to_csv('df_violations.csv') <--- commented out

In [18]:
# phone number is not necessary for our work
# df.drop('PHONE', axis=1, inplace = True)

In [10]:
# create a readable address
df['ADDRESS'] = df['BUILDING'] + ' ' + df['STREET']

In [11]:
# drop the columns that made up the address
df.drop(['BUILDING', 'STREET'], axis=1, inplace = True)

In [13]:
# creates a column 'closed', which will indicate if a restaurant's inspection forced it to close
df['CLOSED'] = df.ACTION.apply(lambda x: 1 if 'Establishment Closed ' in x else 0)

In [14]:
# removes the columns that were added to the dataframe minus CAMIS, which is the unique ID for the restaurant
df.drop(['VIOLATION DESCRIPTION', 'VIOLATION CODE', 'ACTION', 'CRITICAL FLAG'], axis=1, inplace = True)

In [16]:
# creates a day of the week column, but also converts the inspection_date column
# to the day of the week
day_of_week = [None] * len(df['inspection_date'])
for i in range(len(df['inspection_date'])):
    dt = df['inspection_date'][i]
    year, month, day = (int(x) for x in dt.split('-'))    
    answer = datetime.date(year, month, day).weekday()
    if answer == 0:
        day_of_week[i] = 'Monday'
    if answer == 1:
        day_of_week[i] = 'Tuesday'
    if answer == 2:
        day_of_week[i] = 'Wednesday'
    if answer == 3:
        day_of_week[i] = 'Thursday'
    if answer == 4:
        day_of_week[i] = 'Friday'
    if answer == 5:
        day_of_week[i] = 'Saturday'
    if answer == 6:
        day_of_week[i] = 'Sunday'
df['DAY OF WEEK'] = day_of_week

In [18]:
df.drop(['GRADE DATE'], axis=1, inplace=True)

In [20]:
df.head(2)

,CAMIS,DBA,BORO,ZIPCODE,PHONE,CUISINE DESCRIPTION,SCORE,GRADE,INSPECTION TYPE,inspection_date,ADDRESS,CLOSED,DAY OF WEEK
0,30075445,MORRIS PARK BAKE SHOP,BRONX,10462.0,7188924968,Bakery,18.0,NaN,Cycle Inspection / Initial Inspection,2019-05-16,1007 MORRIS PARK AVE,0,Thursday
1,30075445,MORRIS PARK BAKE SHOP,BRONX,10462.0,7188924968,Bakery,18.0,NaN,Cycle Inspection / Initial Inspection,2019-05-16,1007 MORRIS PARK AVE,0,Thursday


In [21]:
col = df.columns

In [22]:
col

Index(['CAMIS', 'DBA', 'BORO', 'ZIPCODE', 'PHONE', 'CUISINE DESCRIPTION',
       'SCORE', 'GRADE', 'INSPECTION TYPE', 'inspection_date', 'ADDRESS',
       'CLOSED', 'DAY OF WEEK'],
      dtype='object')

In [23]:
df_unique = pd.DataFrame(columns=col)

In [24]:
df_unique

,CAMIS,DBA,BORO,ZIPCODE,PHONE,CUISINE DESCRIPTION,SCORE,GRADE,INSPECTION TYPE,inspection_date,ADDRESS,CLOSED,DAY OF WEEK


In [25]:
df_unique = df.drop_duplicates(subset=['CAMIS', 'inspection_date'], keep='first')

In [28]:
df_unique.head(2)

,CAMIS,DBA,BORO,ZIPCODE,PHONE,CUISINE DESCRIPTION,SCORE,GRADE,INSPECTION TYPE,inspection_date,ADDRESS,CLOSED,DAY OF WEEK
0,30075445,MORRIS PARK BAKE SHOP,BRONX,10462.0,7188924968,Bakery,18.0,NaN,Cycle Inspection / Initial Inspection,2019-05-16,1007 MORRIS PARK AVE,0,Thursday
5,30075445,MORRIS PARK BAKE SHOP,BRONX,10462.0,7188924968,Bakery,5.0,A,Cycle Inspection / Initial Inspection,2018-05-11,1007 MORRIS PARK AVE,0,Friday


In [297]:
for i in df_unique['INSPECTION TYPE'].unique():
    print(i, df_unique[df_unique['INSPECTION TYPE'].str.contains(i, regex=True)].isnull().sum())

Cycle Inspection / Initial Inspection CAMIS                  0    
DBA                    0    
BORO                   0    
ZIPCODE                864  
CUISINE DESCRIPTION    0    
SCORE                  0    
GRADE                  25023
INSPECTION TYPE        0    
inspection_date        0    
ADDRESS                40   
closed                 0    
DAY OF WEEK            0    
dtype: int64
Cycle Inspection / Re-inspection CAMIS                  0  
DBA                    0  
BORO                   0  
ZIPCODE                339
CUISINE DESCRIPTION    0  
SCORE                  0  
GRADE                  515
INSPECTION TYPE        0  
inspection_date        0  
ADDRESS                8  
closed                 0  
DAY OF WEEK            0  
dtype: int64
Cycle Inspection / Compliance Inspection CAMIS                  0  
DBA                    0  
BORO                   0  
ZIPCODE                1  
CUISINE DESCRIPTION    0  
SCORE                  0  
GRADE                  172
I

/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


Pre-permit (Non-operational) / Initial Inspection CAMIS                  0
DBA                    0
BORO                   0
ZIPCODE                0
CUISINE DESCRIPTION    0
SCORE                  0
GRADE                  0
INSPECTION TYPE        0
inspection_date        0
ADDRESS                0
closed                 0
DAY OF WEEK            0
dtype: int64
Pre-permit (Operational) / Second Compliance Inspection CAMIS                  0
DBA                    0
BORO                   0
ZIPCODE                0
CUISINE DESCRIPTION    0
SCORE                  0
GRADE                  0
INSPECTION TYPE        0
inspection_date        0
ADDRESS                0
closed                 0
DAY OF WEEK            0
dtype: int64
Pre-permit (Non-operational) / Second Compliance Inspection CAMIS                  0
DBA                    0
BORO                   0
ZIPCODE                0
CUISINE DESCRIPTION    0
SCORE                  0
GRADE                  0
INSPECTION TYPE        0
inspecti

In [314]:
df_unique[df_unique['INSPECTION TYPE'].str.contains('Cycle Inspection / Initial Inspection', regex=True)].isnull().sum()

CAMIS                  0    
DBA                    0    
BORO                   0    
ZIPCODE                864  
CUISINE DESCRIPTION    0    
SCORE                  0    
GRADE                  25023
INSPECTION TYPE        0    
inspection_date        0    
ADDRESS                40   
closed                 0    
DAY OF WEEK            0    
dtype: int64

In [327]:
df_unique[df_unique['GRADE'] == 'Z']

,CAMIS,DBA,BORO,ZIPCODE,CUISINE DESCRIPTION,SCORE,GRADE,INSPECTION TYPE,inspection_date,ADDRESS,closed,DAY OF WEEK
844,40365499,YANKEE TAVERN,BRONX,10451.0,American,25.0,Z,Cycle Inspection / Re-inspection,2019-03-11,72 EAST 161 STREET,0,Monday
896,40365577,V & T RESTAURANT,MANHATTAN,10025.0,Italian,22.0,Z,Cycle Inspection / Re-inspection,2019-02-20,1024 AMSTERDAM AVENUE,0,Wednesday
1327,40366586,SHIMONS KOSHER PIZZA,QUEENS,11367.0,Jewish/Kosher,40.0,Z,Cycle Inspection / Re-inspection,2019-05-15,7124 MAIN STREET,0,Wednesday
1543,40367005,DA VINCI PIZZA,BROOKLYN,11204.0,Pizza/Italian,27.0,Z,Cycle Inspection / Re-inspection,2019-05-29,6514 18 AVENUE,0,Wednesday
1769,40367540,PICCOLA VENEZIA,QUEENS,11103.0,Italian,96.0,Z,Cycle Inspection / Re-inspection,2019-05-11,4201 28 AVENUE,0,Saturday
1997,40368271,CHERRY TAVERN,MANHATTAN,10009.0,American,18.0,Z,Cycle Inspection / Reopening Inspection,2019-04-08,441 EAST 6 STREET,0,Monday
2204,40369012,VALENTINO'S PIZZA,QUEENS,11367.0,Pizza/Italian,7.0,Z,Cycle Inspection / Reopening Inspection,2019-03-06,7147 KISSENA BOULEVARD,0,Wednesday
2268,40369087,BLUE BAY RESTAURANT,BRONX,10463.0,American,47.0,Z,Cycle Inspection / Re-inspection,2019-05-20,3533 JOHNSON AVENUE,0,Monday
2571,40369878,VILLA MOSCONI RESTAURANT,MANHATTAN,10012.0,Italian,19.0,Z,Cycle Inspection / Re-inspection,2019-05-16,69 MACDOUGAL STREET,0,Thursday
3199,40373938,IHOP,BRONX,10463.0,American,26.0,Z,Cycle Inspection / Re-inspection,2019-03-28,5655 BROADWAY,0,Thursday


In [33]:
df['ZIP'] = df.ZIPCODE.apply(lambda x: df.ZIPCODE[x].as_type(int) if df.ZIPCODE[x] == True in df else '')

TypeError: cannot do label indexing on <class 'pandas.core.indexes.range.RangeIndex'> with these indexers [nan] of <class 'float'>

In [65]:
for i in range(len(df.ZIPCODE)):
    if df.ZIPCODE[i] != None or df.ZIPCODE[i] == '':
        df.ZIPCODE[i] = df.ZIPCODE[i][:-2].astype(int)
        if i % 100 == 0:
            print(i)
    else:
        df.ZIPCODE[i] = 'Missing'

TypeError: 'int' object is not subscriptable

In [54]:
type(df.ZIPCODE[0])

str

In [41]:
for i in range(len(df.ZIPCODE)):
    if df.ZIPCODE[i] is 'NaN':
        df.ZIP[i] = 'N/A'
    else:
        df.ZIP[i] = int(df.ZIPCODE[i])
        if i % 100 == 0:
            print(i)

/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


0
100
200


ValueError: cannot convert float NaN to integer

In [30]:
df['ZIP']

0          
1          
2          
3          
4          
5          
6          
7          
8          
9          
10         
11         
12         
13         
14         
15         
16         
17         
18         
19         
20         
21         
22         
23         
24         
25         
26         
27         
28         
29         
         ..
268428     
268429     
268430     
268431     
268432     
268433     
268434     
268435     
268436     
268437     
268438     
268439     
268440     
268441     
268442     
268443     
268444     
268445     
268446     
268447     
268448     
268449     
268450     
268451     
268452     
268453     
268454     
268455     
268456     
268457     
Name: ZIP, Length: 268458, dtype: object

In [29]:
rest_ref = df[['PHONE', 'DBA', 'ZIPCODE']]

In [30]:
rest_uni_zip = rest_ref.drop_duplicates(subset=['PHONE', 'DBA'], keep='first')

In [31]:
pd.set_option('display.max_rows', 1000)

In [34]:
sorted_rest_uni = rest_uni.groupby('DBA').count().sort_values(['PHONE'], ascending=False)

KeyError: 'PHONE'

In [33]:
sorted_rest_uni

,CAMIS
DBA,
DUNKIN',438
SUBWAY,277
STARBUCKS,266
MCDONALD'S,189
"DUNKIN', BASKIN ROBBINS",110
POPEYES,90
BURGER KING,82
KENNEDY FRIED CHICKEN,82
DOMINO'S,74


In [35]:
rest_uni_zip.groupby(['ZIPCODE', 'DBA']).count().sort_values(['PHONE'], ascending=False)

,,PHONE
ZIPCODE,DBA,
10017.0,STARBUCKS,14
10001.0,STARBUCKS,12
10019.0,STARBUCKS,12
10036.0,STARBUCKS,11
11201.0,STARBUCKS,10
10314.0,DUNKIN',10
10022.0,STARBUCKS,10
11101.0,DUNKIN',9
10013.0,STARBUCKS,9
